In [26]:
import pandas as pd
from fastai.text import *

# Loads user given CSV File
def load_data(file_path):
    # Load only the first 1000 rows for testing
    df = pd.read_csv(file_path, encoding='ISO-8859-1', nrows=5000)
    print(f"Loaded data with {len(df)} records.")
    return df

# Preprocesses data for training
def preprocess_data(df):
    # Make sure the CSV has the correct columns for 'text' and 'sentiment'
    df = df[['text', 'sentiment']]
    
    # Create a DataLoader for text classification
    dls = TextDataLoaders.from_df(df, text_col='text', label_col='sentiment', valid_pct=0.2, bs=32)
    print(f"DataLoader created with {len(dls.train)} training samples and {len(dls.valid)} validation samples.")
    return dls

def learn_data(dls):
    # Define text classifier and fine-tune it
    learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
    learn.fine_tune(4, 1e-2)
    
    return learn

# Predict sentiment for each word in a paragraph
def predict_word_sentiment(learn, paragraph):
    # Tokenize the paragraph into individual words
    words = paragraph.split()
    
    # Create a list to store sentiment predictions
    word_sentiments = []
    
    # Get predictions for each word
    for word in words:
        # FastAI's `predict` method expects a list of text, so we pass the word as a list
        pred, _, probs = learn.predict(word)
        
        # Append the predicted sentiment and probability for each word
        word_sentiments.append((word, pred, probs))
    
    return word_sentiments

# Main function to execute the script
def main():
    # Load the CSV file containing tweets and sentiments
    file_path = 'train.csv' 
    df = load_data(file_path)
    
    # Preprocess the data and create DataLoader
    dls = preprocess_data(df)

    dls = learn_data(dls)

    # Test the model with a paragraph of text
    test_paragraph = "I love this place! The food is amazing, and the service is great."
    
    word_sentiments = predict_word_sentiment(dls, test_paragraph)
    
    # Print the word-level sentiments
    for word, sentiment, probs in word_sentiments:
        print(f"Word: {word}, Sentiment: {sentiment}, Probabilities: {probs}")
    

# Execute the script
if __name__ == "__main__":
    main()